In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import rankdata
from scipy.linalg import cholesky
import matplotlib.pyplot as plt

In [24]:
# Step 1: Load data
df = pd.read_csv('2024-11-23-JHU_IDD-hierarchSIM.csv')
df = df[df['horizon']==3].reset_index()[['strain_0', 'strain_1', 'strain_2']]
data = df.values
n,d = data.shape

# Step 2: Compute empirical CDF → Gaussian copula marginals
ranks = np.array([rankdata(data[:, i]) for i in range(d)]).T
u = (ranks - 0.5) / n
z = norm.ppf(u)

# Step 3: Define Gaussian copula sample generator
def sample_gaussian_copula(z_data, corr_matrix, n_samples=1000):
    L = cholesky(corr_matrix, lower=True)
    z_std = np.random.randn(n_samples, d)
    z_copula = z_std @ L.T

    # Transform back via inverse CDF of original marginals
    u_copula = norm.cdf(z_copula)
    new_samples = np.zeros_like(z_copula)

    for i in range(d):
        ecdf_sorted = np.sort(data[:, i])
        ranks = np.floor(u_copula[:, i] * (n - 1)).astype(int)
        new_samples[:, i] = ecdf_sorted[ranks]

    return new_samples

# Step 4: Define correlation matrix with moderately negative correlation
cor_12 = cor_21 = -0.5
cor_13 = cor31 = -0.5
cor_23 = cor_32 = -0.1
corr = np.array([
    [1.0, cor_12, cor_13],
    [cor_21, 1.0, cor_23],
    [cor31, cor_32, 1.0]
])

# Step 5: Sample and compare
copula_samples = sample_gaussian_copula(z, corr, n_samples=n)

# Step 6: Compare quantiles
original_q95 = np.quantile(data, 0.975, axis=0)
copula_q95 = np.quantile(copula_samples, 0.95, axis=0)

comparison = pd.DataFrame({
    "Original 95%": f'{np.quantile(np.sum(data, axis=1), 0.025, axis=0):.0f}-{np.quantile(np.sum(data, axis=1), 0.975, axis=0):.0f}',
    "Copula 95%": f'{np.quantile(np.sum(copula_samples, axis=1), 0.025, axis=0):.0f}-{np.quantile(np.sum(copula_samples, axis=1), 0.975, axis=0):.0f}',
}, index=['Sum(AH1, AH3, B)'])

print(comparison)


                 Original 95% Copula 95%
Sum(AH1, AH3, B)       21-529     63-441
